In [1]:
import glob
import numpy as np
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord

/home/alexm/.conda/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alexm/.conda/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/alexm/.conda/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/alexm/.conda/envs/tf-gpu/lib/python3.6/site-packages

In [2]:
notes = []
for file in glob.glob("data/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts:  # File has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else:  # File has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
    
    print("Finished parsing the following MIDI file:", file)

Finished parsing the following MIDI file: data/Holy-Jazz-Piano.mid
Finished parsing the following MIDI file: data/hyrule-castle-town-in-c-major_piano-duet.mid
Finished parsing the following MIDI file: data/LoZOoT_-_Prelude_of_Light_Piano_Only_-_By_DaDude.mid
Finished parsing the following MIDI file: data/clocktower.mid
Finished parsing the following MIDI file: data/Z-OoA_LynnaCity-PianoOnly.mid
Finished parsing the following MIDI file: data/MajMas-ALIENSOLO.mid
Finished parsing the following MIDI file: data/command_melody_lozww_piano.mid
Finished parsing the following MIDI file: data/Come Rain or Come Shine V1.mid
Finished parsing the following MIDI file: data/Outset_Island-1.mid
Finished parsing the following MIDI file: data/ETPV.mid
Finished parsing the following MIDI file: data/hyrule_field.mid
Finished parsing the following MIDI file: data/Zelda_Overworld.mid
Finished parsing the following MIDI file: data/ordon.mid
Finished parsing the following MIDI file: data/darkwrld.mid
Finishe

In [3]:
np_notes = np.array(notes)
np.save("cached_data/notes.npy", np_notes)

In [4]:
print(np_notes[0:100])

['C4' 'F4' '5.9.0' 'C2' 'C3' '5.9' 'C4' 'F4' 'D4' 'B-4' 'C3' 'F4' 'C5'
 'G#4' 'A4' '10.2.5' 'F4' 'A4' 'B3' 'C4' 'C3' 'F4' 'B-3' 'G4' 'F4' '4.5'
 'F#2' 'F3' 'G2' 'B-3' 'B3' 'F4' 'G1' 'F3' 'B-2' 'B-1' '11' '10.2.5' 'C2'
 'C3' '10.2' '2.5' 'F3' 'C3' 'C2' 'F4' 'G4' 'C5' 'G#4' 'C2' 'C3' 'A4'
 '10.0.2' 'G#4' 'C5' 'B4' 'A4' 'F4' 'C4' 'C3' 'F4' 'B-3' 'B3' 'C3' '2.5'
 'B-3' 'C3' 'F3' 'F2' 'F#2' 'F3' 'G2' 'B-3' 'G1' 'B3' 'F4' 'F3' '10' '11'
 'F3' 'D4' 'B-3' 'C2' 'C3' 'F4' 'C3' '11.0.5' '5' '5' 'C5' 'C#5' 'D5' '5'
 'F4' '5.7' '9.2' '8' '1.4.8' '7.0' 'E-5']


In [5]:
sequence_length = 100
n_vocab = len(set(notes))

# Get all pitch names
pitch_names = sorted(set(item for item in notes))

# Create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitch_names))
network_input = []
network_output = []

# Create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
    
n_patterns = len(network_input)

# Reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

# Normalize input
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)

In [6]:
np_network_input = np.array(network_input)
np.save("cached_data/network_input.npy", np_network_input)
np_network_output = np.array(network_output)
np.save("cached_data/network_output.npy", np_network_output)

In [7]:
print(np_network_input)

[[[0.90559441]
  [0.96853147]
  [0.56468531]
  ...
  [0.12937063]
  [0.63811189]
  [0.93356643]]

 [[0.96853147]
  [0.56468531]
  [0.9020979 ]
  ...
  [0.63811189]
  [0.93356643]
  [0.84965035]]

 [[0.56468531]
  [0.9020979 ]
  [0.90384615]
  ...
  [0.93356643]
  [0.84965035]
  [0.8513986 ]]

 ...

 [[0.91958042]
  [0.88286713]
  [0.49125874]
  ...
  [0.9020979 ]
  [0.98951049]
  [0.94230769]]

 [[0.88286713]
  [0.49125874]
  [0.90909091]
  ...
  [0.98951049]
  [0.94230769]
  [0.87937063]]

 [[0.49125874]
  [0.90909091]
  [0.88286713]
  ...
  [0.94230769]
  [0.87937063]
  [0.91958042]]]


In [8]:
print(np_network_output)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
